# introduction and basic architecture of object detection -> for single object of interest

In [4]:
import tensorflow as tf



resnetModel = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

resnetModel.trainable = False


modelOutput = resnetModel.output
flat = tf.keras.layers.Flatten()(modelOutput)



# add classification head to it -> to classify the object of interest and it's instances to a class
x1 = tf.keras.layers.Dense(128, activation="relu")(flat)
x2 = tf.keras.layers.Dense(64, activation="relu")(x1)
x3 = tf.keras.layers.Dense(32, activation="relu")(x2)
x4 = tf.keras.layers.Dense(3, activation="softmax", name="class_head")(x3)

# this head detects the class of the object


# add regression head (bx, bw, bh, bw)
y1 = tf.keras.layers.Dense(128, activation="relu")(flat)
y2 = tf.keras.layers.Dense(64, activation="relu")(y1)
y3 = tf.keras.layers.Dense(32, activation="relu")(y2)

# for scaled values
#y4 = tf.keras.layers.Dense(4, activation="sigmoid")(y3)

# for absolute values
y4 = tf.keras.layers.Dense(4, name="reg_head")(y3)

# this head detects the bounding box coordinates -> bx, by, bh, bw



modelOut = tf.keras.models.Model(inputs=resnetModel.input, outputs=[y4, x4])
modelOut.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 49,298,983 (188.06 MB)

 Trainable params: 25,711,271 (98.08 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [5]:
losses = {"class_head" : "categorical_crossentropy", "reg_head" : "mse"}
metrics = {"class_head" : "accuracy", "reg_head" : "mse"}

modelOut.compile(loss=losses, optimizer="adam", metrics=metrics)


In [ ]:

# model will take input x and two labes y1 for class (one hot encoding) and y2 for coordinate (4 values)
# x -> arry of images -> n no. of (224, 224, 3) images
# y1 -> class ->  one hot encoding of classes : array of [0, 0, 1]
# y2 -> coordinates ->  four values : array of [3, 4, 5, 1]
model.fit(x, [y1, y2])